#Creating envs

1- sdxl env for the generation model

2-ootd env for the virtual try on model

In [ ]:
# Install virtualenv if not available
!pip install virtualenv -q

# Create SDXL environment (newer libraries)
!virtualenv /content/sdxl_env
# Fix for activation script path
!ls -la /content/sdxl_env/bin/

# Install specific PyTorch version for Stable Diffusion
!. /content/sdxl_env/bin/activate && pip install torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124 -q
# Install specific packages for SD environment
!. /content/sdxl_env/bin/activate && pip install -r/content/requirements_sdxl.txt -q

# Create OOTDiffusion environment (older libraries)
!virtualenv /content/ootd_env

# Install specific PyTorch version for OOTDiffusion
!. /content/ootd_env/bin/activate && pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118 -q
# Install from your existing requirements file
!. /content/ootd_env/bin/activate && pip install -r/content/requirements_ootd.txt -q

In [ ]:
!. /content/ootd_env/bin/activate && pip install git+https://github.com/huggingface/diffusers.git -q

# Cloning OOTDiffusion and checkpoints

In [ ]:
!git lfs install
!git clone https://github.com/levihsu/OOTDiffusion
%cd /content/OOTDiffusion
!git clone https://huggingface.co/openai/clip-vit-large-patch14 checkpoints/clip-vit-large-patch14
!git clone https://huggingface.co/levihsu/OOTDiffusion

# Move openpose,humanparsing and ootd
!mv -n /content/OOTDiffusion/OOTDiffusion/checkpoints/ootd /content/OOTDiffusion/checkpoints
!mv -n /content/OOTDiffusion/OOTDiffusion/checkpoints/humanparsing /content/OOTDiffusion/checkpoints
!mv -n /content/OOTDiffusion/OOTDiffusion/checkpoints/openpose /content/OOTDiffusion/checkpoints
%cd /content/OOTDiffusion/run

#*You need to go to /content/ootd_env/lib/python3.11/site-packages/diffusers/utils/dynamic_modules_utils.py and remove cached_download from the line "from huggingface_hub import HfFolder, cached_download, hf_hub_download, model_info"

In [23]:
%%writefile /content/generate_clothing.py
def generate_clothing_image(
    prompt,
    lora_path=None,
    negative_prompt=None,
    num_steps=30,
    guidance_scale=7.5,
    seed=42
):
    """
    Generates a clothing image using Stable Diffusion 2.1 with optional LoRA weights.

    Args:
        prompt (str): The prompt describing the clothing to generate
        lora_path (str, optional): Path to LoRA adapter weights
        lora_scale (float, optional): Scale factor for LoRA weights
        negative_prompt (str, optional): Negative prompt to guide generation away from certain features
        num_steps (int, optional): Number of inference steps
        guidance_scale (float, optional): Guidance scale for generation
        seed (int, optional): Random seed for reproducibility

    Returns:
        PIL.Image: Generated image
        str: Path where the image was saved
    """
    from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
    import torch
    from safetensors.torch import load_file

    # Default negative prompt if none provided
    if negative_prompt is None:
        negative_prompt = "wrinkled, dirty, worn, text, logo, brand name, person, model, mannequin, low quality, worst quality, blurry"

    # Initialize pipeline with SD 2.1
    pipe = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        torch_dtype=torch.float16,
    ).to("cuda")


    # Load and apply LoRA weights if provided
    if lora_path:
        # Method to load LoRA weights
        pipe.unet.load_attn_procs(lora_path)


    # Enable memory optimizations
    pipe.enable_xformers_memory_efficient_attention()

    # Enhance prompt with plain white background specification
    enhanced_prompt = f"{prompt}, on plain white background"

    image = pipe(
        prompt=enhanced_prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
    ).images[0]

    # Save image
    save_path = "/content/clothing.png"
    image.save(save_path)

    return image, save_path

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="Generate clothing images with SD 2.1 and LoRA")
    parser.add_argument("--prompt", type=str, required=True, help="Prompt for image generation")
    parser.add_argument("--lora_path", type=str, help="Path to LoRA adapter weights")
    parser.add_argument("--negative_prompt", type=str, help="Negative prompt")
    parser.add_argument("--steps", type=int, default=30, help="Number of inference steps")
    parser.add_argument("--guidance", type=float, default=7.5, help="Guidance scale")
    parser.add_argument("--seed", type=int, default=42, help="Random seed")

    args = parser.parse_args()

    image, path = generate_clothing_image(
        prompt=args.prompt,
        lora_path=args.lora_path,
        negative_prompt=args.negative_prompt,
        num_steps=args.steps,
        guidance_scale=args.guidance,
        seed=args.seed
    )

    print(f"Image saved to {path}")

Overwriting /content/generate_clothing.py


# Generating clothes and fitting model

In [ ]:
!source /content/sdxl_env/bin/activate && python /content/generate_clothing.py --prompt "simple grey tshirt" \
 --lora_path "/content/pytorch_lora_weights.safetensors" --steps 30 --guidance 7.5

In [ ]:
# change model path and cloth path,
#Garment category must be paired: 0 = upperbody; 1 = lowerbody; 2 = dress
# Step 2: Use the generated image with OOTDiffusion
!source /content/ootd_env/bin/activate && python run_ootd.py --model_path /content/vt_test.jpg --cloth_path /content/clothing.png --model_type dc --category 0 --scale 2.0 --sample 4